# 8. OpenAI Responses Target

In this demo, we show an example of the `OpenAIResponseTarget`. [Responses](https://platform.openai.com/docs/api-reference/responses) is a newer protocol than chat completions and provides additional functionality with a somewhat modified API. The allowed input types include text, image, web search, file search, functions, reasoning, and computer use.

Before you begin, ensure you are set up with the correct version of PyRIT installed and have secrets configured as described [here](../../setup/populating_secrets.md).


## OpenAI Configuration

Like most targets, all `OpenAITarget`s need an `endpoint` and often also needs a `model` and a `key`. These can be passed into the constructor or configured with environment variables (or in .env).

- endpoint: The API endpoint (`OPENAI_RESPONSES_ENDPOINT` environment variable). For OpenAI, these are just "https://api.openai.com/v1/responses".
- auth: The API key for authentication (`OPENAI_RESPONSES_KEY` environment variable).
- model_name: The model to use (`OPENAI_RESPONSES_MODEL` environment variable). For OpenAI, these are any available model name and are listed here: "https://platform.openai.com/docs/models".

In [1]:
from pyrit.executor.attack import ConsoleAttackResultPrinter, PromptSendingAttack
from pyrit.prompt_target import OpenAIResponseTarget
from pyrit.setup import IN_MEMORY, initialize_pyrit

initialize_pyrit(memory_db_type=IN_MEMORY)

target = OpenAIResponseTarget()
# For an AzureOpenAI endpoint with Entra ID authentication enabled, use the following command instead. Make sure to run `az login` first.
# target = OpenAIResponseTarget(use_entra_auth=True)

attack = PromptSendingAttack(objective_target=target)

result = await attack.execute_async(objective="Tell me a joke")  # type: ignore
await ConsoleAttackResultPrinter().print_conversation_async(result=result)  # type: ignore


────────────────────────────────────────────────────────────────────────────────────────────────────
🔹 Turn 1 - USER
────────────────────────────────────────────────────────────────────────────────────────────────────
  Tell me a joke

────────────────────────────────────────────────────────────────────────────────────────────────────
🔸 ASSISTANT
────────────────────────────────────────────────────────────────────────────────────────────────────
  Why don’t skeletons fight each other?
    They don’t have the guts!

────────────────────────────────────────────────────────────────────────────────────────────────────


## Tool Use with Custom Functions

In this example, we demonstrate how the OpenAI `Responses API` can be used to invoke a **custom-defined Python function** during a conversation. This is part of OpenAI’s support for "function calling", where the model decides to call a registered function, and the application executes it and passes the result back into the conversation loop.

We define a simple tool called `get_current_weather`, which simulates weather information retrieval. A corresponding OpenAI tool schema describes the function name, parameters, and expected input format.

The function is registered in the `custom_functions` argument of `OpenAIResponseTarget`. The `extra_body_parameters` include:

- `tools`: the full OpenAI tool schema for `get_current_weather`.
- `tool_choice: "auto"`: instructs the model to decide when to call the function.

The user prompt explicitly asks the model to use the `get_current_weather` function. Once the model responds with a `function_call`, PyRIT executes the function, wraps the output, and the conversation continues until a final answer is produced.

This showcases how agentic function execution works with PyRIT + OpenAI Responses API.

In [2]:
from pyrit.models import Message, MessagePiece
from pyrit.setup import IN_MEMORY, initialize_pyrit

initialize_pyrit(memory_db_type=IN_MEMORY)


async def get_current_weather(args):
    return {
        "weather": "Sunny",
        "temp_c": 22,
        "location": args["location"],
        "unit": args["unit"],
    }


# Responses API function tool schema (flat, no nested "function" key)
function_tool = {
    "type": "function",
    "name": "get_current_weather",
    "description": "Get the current weather in a given location",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {"type": "string", "description": "City and state"},
            "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
        },
        "required": ["location", "unit"],
        "additionalProperties": False,
    },
    "strict": True,
}

# Let the model auto-select tools
target = OpenAIResponseTarget(
    custom_functions={"get_current_weather": get_current_weather},
    extra_body_parameters={
        "tools": [function_tool],
        "tool_choice": "auto",
    },
    httpx_client_kwargs={"timeout": 60.0},
)

# Build the user prompt
message_piece = MessagePiece(
    role="user",
    original_value="What is the weather in Boston in celsius? Use the get_current_weather function.",
    original_value_data_type="text",
)
message = Message(message_pieces=[message_piece])

response = await target.send_prompt_async(message=message)  # type: ignore

for idx, piece in enumerate(response.message_pieces):
    print(f"{idx} | {piece.role}: {piece.original_value}")

0 | assistant: Currently in Boston, MA:
- Weather: Sunny
- Temperature: 22°C


## Using the Built-in Web Search Tool

In this example, we use a built-in PyRIT helper function `web_search_tool()` to register a web search tool with OpenAI's Responses API. This allows the model to issue web search queries during a conversation to supplement its responses with fresh information.

The tool is added to the `extra_body_parameters` passed into the `OpenAIResponseTarget`. As before, `tool_choice="auto"` enables the model to decide when to invoke the tool.

The user prompt asks for a recent positive news story — an open-ended question that may prompt the model to issue a web search tool call. PyRIT will automatically execute the tool and return the output to the model as part of the response.

This example demonstrates how retrieval-augmented generation (RAG) can be enabled in PyRIT through OpenAI's Responses API and integrated tool schema.

NOTE that web search is NOT supported through an Azure OpenAI endpoint, only through the OpenAI platform endpoint (i.e. api.openai.com)

In [3]:
import os

from pyrit.common.tool_configs import web_search_tool
from pyrit.models import Message, MessagePiece
from pyrit.prompt_target.openai.openai_response_target import OpenAIResponseTarget
from pyrit.setup import IN_MEMORY, initialize_pyrit

initialize_pyrit(memory_db_type=IN_MEMORY)

# Note: web search is not yet supported by Azure OpenAI endpoints so we'll use OpenAI from here on.
target = OpenAIResponseTarget(
    endpoint=os.getenv("PLATFORM_OPENAI_RESPONSES_ENDPOINT"),
    api_key=os.getenv("PLATFORM_OPENAI_RESPONSES_KEY"),
    model_name=os.getenv("PLATFORM_OPENAI_RESPONSES_MODEL"),
    extra_body_parameters={
        "tools": [web_search_tool()],
        "tool_choice": "auto",
    },
    httpx_client_kwargs={"timeout": 60},
)

message_piece = MessagePiece(
    role="user", original_value="Briefly, what is one positive news story from today?", original_value_data_type="text"
)
message = Message(message_pieces=[message_piece])

response = await target.send_prompt_async(message=message)  # type: ignore

for idx, piece in enumerate(response.message_pieces):
    if piece.original_value_data_type != "reasoning":
        print(f"{idx} | {piece.role}: {piece.original_value}")

2 | assistant: {"type":"web_search_call","id":"ws_04827fa7a031f2b300692638681710819fb540bd365ab84ac8"}
4 | assistant: One bright spot today: Big Brothers Big Sisters of Orange County & the Inland Empire officially opened “Good Company OC,” a collaborative coworking space in Santa Ana that’s available—rent-free—to local non-profits and community groups, aiming to boost capacity, partnerships, and impact across the region. ([vannuysnewspress.com](https://www.vannuysnewspress.com/big-brothers-big-sisters-of-oc-ie-creates-good-company-oc-collaborative-workspaces-open-to-area-non-profits-and-others/?utm_source=openai))


## Grammar-Constrained Generation

OpenAI models also support constrained generation in the [Responses API](https://platform.openai.com/docs/guides/function-calling#context-free-grammars). This forces the LLM to produce output which conforms to the given grammar, which is useful when specific syntax is required in the output.

In this example, we will define a simple Lark grammar which prevents the model from giving a correct answer to a simple question, and compare that to the unconstrained model.

Note that as of October 2025, this is only supported by OpenAI (not Azure) on "gpt-5"

In [4]:
from pyrit.setup import IN_MEMORY, initialize_pyrit

initialize_pyrit(memory_db_type=IN_MEMORY)


message_piece = MessagePiece(
    role="user",
    original_value="What is the capital of Italy?",
    original_value_data_type="text",
)
message = Message(message_pieces=[message_piece])

# Define a grammar that prevents "Rome" from being generated
lark_grammar = r"""
start: "I think that it is " SHORTTEXT
SHORTTEXT: /[^RrOoMmEe]{1,8}/
"""

grammar_tool = {
    "type": "custom",
    "name": "CitiesGrammar",
    "description": "Constrains generation.",
    "format": {
        "type": "grammar",
        "syntax": "lark",
        "definition": lark_grammar,
    },
}

target = OpenAIResponseTarget(
    endpoint=os.getenv("PLATFORM_OPENAI_RESPONSES_ENDPOINT"),
    api_key=os.getenv("PLATFORM_OPENAI_RESPONSES_KEY"),
    model_name="gpt-5",
    extra_body_parameters={"tools": [grammar_tool], "tool_choice": "required"},
    temperature=1.0,
)

unconstrained_target = OpenAIResponseTarget(
    endpoint=os.getenv("PLATFORM_OPENAI_RESPONSES_ENDPOINT"),
    api_key=os.getenv("PLATFORM_OPENAI_RESPONSES_KEY"),
    model_name="gpt-5",
    temperature=1.0,
)

unconstrained_result = await unconstrained_target.send_prompt_async(message=message)  # type: ignore

result = await target.send_prompt_async(message=message)  # type: ignore

print("Unconstrained Response:")
for idx, piece in enumerate(unconstrained_result.message_pieces):
    if piece.original_value_data_type != "reasoning":
        print(f"{idx} | {piece.role}: {piece.original_value}")

print()

print("Constrained Response:")
for idx, piece in enumerate(result.message_pieces):
    if piece.original_value_data_type != "reasoning":
        print(f"{idx} | {piece.role}: {piece.original_value}")

Unconstrained Response:
1 | assistant: Rome.

Constrained Response:
1 | assistant: I think that it is x
